In [19]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [20]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-11-10 11:21:54--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4350 (4.2K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   4.25K  --.-KB/s    in 0s      

2025-11-10 11:21:54 (32.3 MB/s) - ‘minsearch.py.1’ saved [4350/4350]



In [22]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/workspaces/llm-zoomcamp/02-vector-search/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [23]:
from openai import OpenAI

client = OpenAI()

In [24]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [26]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-5-nano',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [27]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
rag('how do I run kafka?')

'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java'

In [18]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even if it has already started. You are also eligible to submit the homework assignments. However, make sure to pay attention to the deadlines for turning in the final projects to avoid leaving everything until the last minute.'

In [29]:
## RAG WITH VECTOR SEARCH

In [30]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient("http://localhost:6333")
qd_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='zoomcamp-faq'), CollectionDescription(name='zoomcamp-rag')])

In [31]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [32]:
collection_name = "zoomcamp-faq"

In [33]:
qd_client.delete_collection(collection_name=collection_name)

True

In [34]:
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [35]:
points = []

for i, doc in enumerate(documents):

    id = i
    text = doc['question'] + ' ' + doc['text'] 
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=id,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [40]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [41]:
question = 'the course has already started, can I still enroll?'

In [64]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [66]:
def vector_search(question):
    
    course = 'data-engineering-zoomcamp' 
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( 
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True 
    )
    
    results = []
    
    for point in query_points.points: 
        results.append(point.payload)
    
    return results

In [69]:
def rag_vs(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [70]:
rag_vs('how do i run kafka in my computer?')

'Here are the steps from the course context to run Kafka on your computer (Java setup is shown; there are also Python notes if you’re using Python):\n\n- Start the Kafka broker(s)\n  - If you’re using Docker, make sure the broker containers are running. If you see NoBrokersAvailable, it’s likely the broker container isn’t up.\n  - Use docker ps to check. If needed, start all containers with docker compose up -d in the docker compose folder.\n\n- Configure the client\n  - In your Kafka client scripts, ensure the bootstrap server URL is correct in StreamsConfig.BOOTSTRAP_SERVERS_CONFIG.\n  - Update the cluster credentials in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KAFKA_CLUSTER_SECRET).\n\n- Run the producer/consumer from the terminal (Java)\n  - In the project directory, run:\n    - java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n  - For the consumer, use the analogous command with JsonConsumer.java.\n\n- If you enco